In [1]:
from pdbfixer import PDBFixer
from pathlib import Path
import mdtraj as md
import openmm as mm 
from openmm import app
from openmm import unit as U
import nglview as view

In [13]:
struc = PDBFixer("../data/2efv_fixed.pdb")
struc.findMissingResidues()
struc.findMissingAtoms()
struc.addMissingAtoms()
struc.addMissingHydrogens()
top = struc.topology
pos = struc.positions

model = app.Modeller(top, pos)

topology = model.getTopology()
box_vectors = topology.getPeriodicBoxVectors()

In [14]:
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')

model.addSolvent(
    forcefield,
    padding=1.*U.nanometer,
    model='tip3p',
    ionicStrength=0.1 * U.millimolar, 
    positiveIon='Na+',
    negativeIon='Cl-'
)

# Write model to the pdb file
with open('../data/prototype1/setup.pdb', 'wt') as f:
    app.PDBFile.writeFile(model.getTopology(), model.getPositions(), f)


system = forcefield.createSystem(
    model.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0*U.nanometers,
    constraints=app.HBonds,
    ewaldErrorTolerance=0.0005
)

In [15]:
with open('../data/prototype1.xml', 'w') as f:
    system_xml = mm.XmlSerializer.serialize(system)
    f.write(system_xml) 

In [16]:
help(mm.openmm.LangevinIntegrator)

Help on class LangevinIntegrator in module openmm.openmm:

class LangevinIntegrator(Integrator)
 |  LangevinIntegrator(*args)
 |  
 |  This is an Integrator which simulates a System using Langevin dynamics.
 |  
 |  Method resolution order:
 |      LangevinIntegrator
 |      Integrator
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args)
 |      __init__(self, temperature, frictionCoeff, stepSize) -> LangevinIntegrator
 |      __init__(self, other) -> LangevinIntegrator
 |      Create a LangevinIntegrator.
 |      
 |      Parameters
 |      ----------
 |      temperature : double
 |          the temperature of the heat bath (in Kelvin)
 |      frictionCoeff : double
 |          the friction coefficient which couples the system to the heat bath (in inverse picoseconds)
 |      stepSize : double
 |          the step size with which to integrate the system (in picoseconds)
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  getFriction(self)
 |      getFriction

In [17]:
integrator = mm.openmm.LangevinIntegrator(
    300*U.kelvin, 
    1/U.picosecond, 
    0.002 * U.picoseconds
)
platform = mm.Platform.getPlatformByName("CPU")
simulation = app.simulation.Simulation(
    model.topology,
    system,
    integrator,
    platform
)

simulation.context.setPositions(model.positions)

In [18]:
simulation.reporters.append(app.pdbreporter.PDBReporter("../data/prototype1/run1.pdb", 50))
simulation.reporters.append(app.dcdreporter.DCDReporter('../data/prototype1/run1.dcd', 10))
simulation.reporters.append(app.statedatareporter.StateDataReporter('mm.log', 10, step=True, potentialEnergy=True, temperature=True))

In [19]:
simulation.minimizeEnergy()

In [20]:
### Get out structure after minimization (Bonus trick)
current_reporter = app.pdbreporter.PDBReporter("../data/prototype1/structure_after_minimization.pdb",1)
current_reporter.report(simulation, simulation.context.getState(getPositions=True))

In [21]:
minimized = md.load_pdb("../data/prototype1/structure_after_minimization.pdb")
view.show_mdtraj(minimized)

NGLWidget()

In [22]:
n_steps = 10000
simulation.step(n_steps)

In [23]:
traj1 = md.load_pdb("../data/prototype1/run1.pdb")
view.show_mdtraj(traj1)

NGLWidget(max_frame=199)

In [33]:
base = md.load_pdb("../data/2efv.pdb")
print(list(base.topology.chains))
view.show_mdtraj(base)


[<mdtraj.core.topology.Chain object at 0x15f4a5a90>, <mdtraj.core.topology.Chain object at 0x15f4a5af0>]


NGLWidget()

In [26]:
base = md.load_pdb("../data/2efv_fixed.pdb")
view.show_mdtraj(base)

NGLWidget()

In [27]:
base.topology

<mdtraj.Topology with 1 chains, 92 residues, 762 atoms, 769 bonds at 0x160e9c820>